In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from templates import *

In [5]:
device = 'cuda:0'
conf = ffhq256_autoenc()
# print(conf.name)
model = LitModel(conf)
state = torch.load(f'checkpoints/{conf.name}/last.ckpt', map_location='cpu')
model.load_state_dict(state['state_dict'], strict=False)
model.ema_model.eval()
model.ema_model.to(device);

Global seed set to 0


Model params: 160.69 M


In [3]:
img_dir = 'imgs_align'
for i,file in enumerate(os.listdir(img_dir)):
    print(f'{i}: {file}')

0: idPhoto.png
1: megane.png
2: sample01.png
3: sandy.png
4: women_default.png
5: yotaka_default.png
6: yotaka_smile.png


## 1枚処理

In [8]:
%%time
data = ImageDataset(img_dir, image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False)
batch = data[1]['img'][None]
print("cond")
%time cond = model.encode(batch.to(device))
print("\nencode")
%time xT = model.encode_stochastic(batch.to(device), cond, T=250)
print("\ndecode")
%time pred = model.render(xT, cond, T=20)
print("\ntotal")

cond
CPU times: total: 0 ns
Wall time: 8 ms

encode
CPU times: total: 13.3 s
Wall time: 17.1 s

decode
CPU times: total: 938 ms
Wall time: 1.35 s

total
CPU times: total: 14.3 s
Wall time: 18.5 s


## 複数枚処理

1画像あたり約700MiBくらい使う  
torch.cuda.empty_cache()でメモリを解放すると良い  
!nvidia-smiでメモリを確認できる  
分割した場合1.5倍くらい時間がかかる

### まとめてやった場合

In [ ]:
%%time

data = ImageDataset(img_dir, image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False)

batch = torch.stack([
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],
    data[1]['img'],  
])

print("cond")
%time cond = model.encode(batch.to(device))
torch.cuda.empty_cache()

print("encode")
%time xT = model.encode_stochastic(batch.to(device), cond, T=250)
torch.cuda.empty_cache()

print("decode")
%time pred = model.render(xT, cond, T=20)
torch.cuda.empty_cache()

print("total")

### 分割した場合

In [ ]:
%%time
data = ImageDataset(img_dir, image_size=conf.img_size, exts=['jpg', 'JPG', 'png'], do_augment=False)

xTs = []
conds = []
preds = []
for i in range(10):
    batch = data[1]['img'][None]
    
    print(f"{i+1}")
    print("cond")
    %time cond = model.encode(batch.to(device))
    
    print("encode")
    %time xT = model.encode_stochastic(batch.to(device), cond, T=250)
    
    print("decode")
    %time pred = model.render(xT, cond, T=20)
    
    print()
    conds.append(cond[0])
    xTs.append(xT[0])
    preds.append(pred[0])
    
print("total")

## 画像の保存

In [11]:
from torchvision.utils import save_image

dst_dir = 'imgs_test/imgs_reconstruct/'
file_name = 'megane.png'
dst_path = dst_dir + file_name
if not os.path.exists(dst_dir): os.makedirs(dst_dir)
save_image(pred[0], dst_path)

## 画像の評価

In [121]:
from ssim import *

In [153]:
ori = ori.to(device)
ssim(ori[0][None], pred[0][None])

tensor(0.9804, device='cuda:0')

In [179]:
# import cv2
ori_path = 'imgs_test_align/test01.png'
recon_path = 'imgs_recon/test01.png'
ori = torch.from_numpy(cv2.imread(ori_path).astype(np.float32)).to(device)
recon = torch.from_numpy(cv2.imread(recon_path).astype(np.float32)).to(device)
ori = ori.permute(2, 0, 1)[None]
recon = recon.permute(2, 0, 1)[None]
ssim(ori, recon)

tensor(0.9118, device='cuda:0')